In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math
from config import *

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'Price', 'FPL_pos', 'FPL_points',
       'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == NEXT_GAMEWEEK]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,14.0,FWD,NaN,7.462574
39,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,12.5,MID,NaN,6.876028
82,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,6.8,MID,NaN,6.034037
110,Carlton Morris,4.0,2023-24,Luton,West Ham,1.0,1602.477417,1787.825562,5.5,FWD,NaN,5.843293
162,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,7.6,MID,NaN,5.609241
186,Bruno Borges Fernandes,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,8.5,MID,NaN,5.495123
190,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,5.0,GK,NaN,5.464136
260,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,7.0,MID,NaN,5.217698
347,Bukayo Saka,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,8.7,MID,NaN,5.006219
384,Solly March,4.0,2023-24,Brighton,Newcastle Utd,1.0,1843.758179,1884.512817,6.6,MID,NaN,4.940640


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((454, 12), (49, 12), (162, 12), (193, 12), (50, 12))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,14.0,FWD,NaN,7.462574
39,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,12.5,MID,NaN,6.876028
82,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,6.8,MID,NaN,6.034037
110,Carlton Morris,4.0,2023-24,Luton,West Ham,1.0,1602.477417,1787.825562,5.5,FWD,NaN,5.843293
162,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,7.6,MID,NaN,5.609241


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df_original = df.copy()

In [13]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [14]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [15]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [16]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [17]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Folarin Balogun, FWD, 4.4, 1.68, Arsenal) - (fixtures: 1.0)'

In [18]:
# for p in playersDB:
#     print(p.__str__())

In [19]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [20]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [21]:
budget = 100

In [22]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
        p.benched = True
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [23]:
best_teams = []
best_team = None
best_team_score = 0
best_bench = None

for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team_for_formation = best_full_teams(smallDB, [formation], budget - bench_price)
    
    if best_team_for_formation[0][1] > best_team_score:
        best_team = best_team_for_formation
        best_team_score = best_team_for_formation[0][1]
        best_bench = bench
        
for p in best_team[0][2]:
    p.benched = True
for p in best_bench:
    best_team[0][2].append(p)
        
    

[3, 4, 3] 1 2 1 0
(Matt Turner, GK, 4.0, 0.28, Nott'ham Forest) - (fixtures: 1.0)
(Charlie Taylor, DEF, 4.0, 2.77, Burnley) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.31, Brighton) - (fixtures: 1.0)
(Jakub Moder, MID, 4.4, 1.99, Brighton) - (fixtures: 1.0)
16.3
With formation [3, 4, 3]: 61.59  | (price = 83.39999999999999)
(Guglielmo Vicario, GK, 5.0, 5.46, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 4.92, Liverpool) - (fixtures: 1.0)
(Ethan Pinnock, DEF, 4.5, 4.62, Brentford) - (fixtures: 1.0)
(Ben Mee, DEF, 4.9, 4.59, Brentford) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.88, Liverpool) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 6.03, Brentford) - (fixtures: 1.0)
(Phil Foden, MID, 7.6, 5.61, Manchester City) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 8.5, 5.5, Manchester Utd) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.0, 7.46, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.84, Luton) - (fixtures: 1.0)
(Yoane Wissa, FWD, 6.2, 4.68, Brent

In [24]:
for p in best_team[0][2]:
    print(p.__str__())

(Guglielmo Vicario, GK, 5.0, 5.46, Tottenham) - (fixtures: 1.0)
(William Saliba, DEF, 5.2, 4.78, Arsenal) - (fixtures: 1.0)
(Ethan Pinnock, DEF, 4.5, 4.62, Brentford) - (fixtures: 1.0)
(Ben Mee, DEF, 4.9, 4.59, Brentford) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.88, Liverpool) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 6.03, Brentford) - (fixtures: 1.0)
(Phil Foden, MID, 7.6, 5.61, Manchester City) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 8.5, 5.5, Manchester Utd) - (fixtures: 1.0)
(Jarrod Bowen, MID, 7.0, 5.22, West Ham) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.0, 7.46, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.84, Luton) - (fixtures: 1.0)
(Matt Turner, GK, 4.0, 0.28, Nott'ham Forest) - (fixtures: 1.0)
(Charlie Taylor, DEF, 4.0, 2.77, Burnley) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.31, Brighton) - (fixtures: 1.0)
(Folarin Balogun, FWD, 4.4, 1.68, Arsenal) - (fixtures: 1.0)


In [25]:
%%time
# best_team = best_full_teams(smallDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
f"Best team score: {best_team[0][1]}"

'Best team score: 61.989999999999995'

In [27]:
best_team[0]

([3, 5, 2],
 61.989999999999995,
  <knapsack_problem.player.Player at 0x23b4820e850>])

In [28]:
def best_team_to_df(best_team_squad, df):
    names = [p.name for p in best_team_squad]
    print(names)
    custom_fpl_order = ["FWD", "MID", "DEF", "GK"]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    output = df[df["Name_original"].isin(names)].reset_index()
    output["First_eleven"] = True
    output.loc[11:15, "First_eleven"] = False
    output = output.sort_values(by=["First_eleven", "FPL_pos", "Price"], ascending=False)
    
    return output

In [29]:
best_squad = best_team_to_df(best_team[0][2], df_original)
best_squad

['Guglielmo Vicario', 'William Saliba', 'Ethan Pinnock', 'Ben Mee', 'Mohamed Salah', 'Bryan Mbeumo', 'Phil Foden', 'Bruno Borges Fernandes', 'Jarrod Bowen', 'Erling Haaland', 'Carlton Morris', 'Matt Turner', 'Charlie Taylor', 'Tariq Lamptey', 'Folarin Balogun']


,index,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction,First_eleven
6,190,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,5.0,GK,NaN,5.464136,True
8,489,William Saliba,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,5.2,DEF,NaN,4.777668,True
10,645,Ben Mee,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,4.9,DEF,NaN,4.588961,True
9,612,Ethan Pinnock,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,4.5,DEF,NaN,4.619450,True
1,39,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,12.5,MID,NaN,6.876028,True
5,186,Bruno Borges Fernandes,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,8.5,MID,NaN,5.495123,True
4,162,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,7.6,MID,NaN,5.609241,True
7,260,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,7.0,MID,NaN,5.217698,True
2,82,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,6.8,MID,NaN,6.034037,True
0,9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,14.0,FWD,NaN,7.462574,True


In [30]:
best_squad.to_csv("predictions/best_squad.csv")